In [1]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [2]:
def create_num_cnt(num):
    return [int((num-num%255)/255),255,num%255]
proposal = "Helly world, This is my first proposal ever. Do you think it will work?"
# proposal = proposal.encode(encoding = 'UTF-8', errors = 'strict')
choices = ["No, Not at all", "Yes, Ofccourse", "Yes, Eventually", "Clueless", "hello world"]


proposal += '\0' * (500-len(proposal));

choices = [choice + '\0'* (30-len(choice)) for choice in choices]
# print ("length of {0} is {1}".format(repr(mystring), len(mystring)))



# len(proposal)

In [3]:
program_id = DAO_PROGRAM_ID
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

In [4]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [5]:
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))


In [25]:
accountSchema = CStruct(
    "proposal_id" / U32
)

(proposals_pda, proposals_pda_bump) = PublicKey.find_program_address([b"Dsol_Dao_Governance"], program_id)

proposals = getAccountInfo(client,proposals_pda)
# proposals.proposal_id += 1
proposals

Container(proposal_id=1)

In [12]:
proposal_pda, _ = PublicKey.find_program_address([b"Dsol_Dao_Governance", proposals.proposal_id.to_bytes(4,"little") ], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
governor_account_meta = AccountMeta(PublicKey("J3ACcSXB2ACbX2CL4FWxvZh81KRQt4AtF3XhyvpxxrJK"), False, False)
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(governor_account_meta.pubkey)], metadata_program_pubkey)



payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
proposals_account_meta = AccountMeta(proposals_pda, False, True)
proposal_pda_meta = AccountMeta(proposal_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
metadata_account_meta = AccountMeta(metadata_pda, False, True)
governor_associated_meta = AccountMeta(get_associated_token_address(payer_keypair.public_key, governor_account_meta.pubkey), False, False)



In [13]:
accounts = [
    payer_account_meta,
    governor_account_meta,
    proposals_account_meta,
    proposal_pda_meta,
    sysvar_clock_meta,
    metadata_account_meta,
    governor_associated_meta,

    sys_program_meta
]

proposal_pda_meta


AccountMeta(pubkey=CEoowYFSrbtiSMiJaMaJBVo8Vipg63xoHcGi33W9akZ7, is_signer=False, is_writable=True)

In [14]:
instruction_data = bytes([1])

instruction_data += bytes(proposal, "ascii")
for choice in choices:
    instruction_data += bytes(choice, "ascii")

instruction_data += bytes(create_num_cnt(65279))
# instruction_data

In [15]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    instruction_data
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/3WAQSLDe5X6aFHPVd2xLSCF51LaMhf4hkRvMiqzh1FEuTAnEqZqGkKGvEFup15eSj5fTKNfLVv5NP5MyzjN1sPgc?cluster=devnet'

In [16]:
ProposalResult = CStruct(
    "leading_choice_bump" / U8,
    "choices" / U32[5],
    
)


Vote = CStruct(
    "proposal_pda" / U8[32],
   "mint_id" / U8[32],
    "choice_bumps" / U8[5]
)

accountSchema = CStruct(
    "proposal_pda" / U8[32],
    "proposal" / String,
    "choices" / String,
    "max_vote_bump" / U32,
    "proposal_governor_id" / U8[32],
    "duration" / U64,
    "ending_data" / U64,
    "votes" / Vec(Vote),
    "result" / ProposalResult

)

In [17]:
Proposal = getAccountInfo(client, proposal_pda)

In [18]:
choices = [Proposal.choices[i*30 : i*30 + 30].strip('\0') for i in range(5)]
proposal = Proposal.proposal.strip("\0")
[Proposal.max_vote_bump, proposal, Proposal.votes, choices, Proposal.result]

[0,
 'Helly world, This is my first proposal ever. Do you think it will work?',
 ListContainer([]),
 ['No, Not at all',
  'Yes, Ofccourse',
  'Yes, Eventually',
  'Clueless',
  'hello world'],
 Container(leading_choice_bump=0, choices=ListContainer([0, 0, 0, 0, 0]))]

In [22]:
vote_pda, _ = PublicKey.find_program_address([b"Dsol_Dao_Governance", bytes(proposal_pda_meta.pubkey), Proposal.max_vote_bump.to_bytes(4, "big")], program_id)

governor_account_meta = AccountMeta(PublicKey("J3ACcSXB2ACbX2CL4FWxvZh81KRQt4AtF3XhyvpxxrJK"), False, False)
governor_pda, _ = PublicKey.find_program_address([b"Dsol_Dao_Governance", bytes(proposal_pda_meta.pubkey), bytes(governor_account_meta.pubkey)], program_id)


metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(governor_account_meta.pubkey)], metadata_program_pubkey)


governor_associated_meta = AccountMeta(get_associated_token_address(payer_keypair.public_key, governor_account_meta.pubkey), False, False)

accounts = [
    payer_account_meta,
    governor_account_meta,
    # AccountMeta(vote_pda, False, True),
    proposal_pda_meta,
    AccountMeta(governor_pda, False, True),
    AccountMeta(metadata_pda, False, False),
    governor_associated_meta,

    sys_program_meta,
    sys_program_meta
]

In [23]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([2,2,110,240,13,200] + create_num_cnt(Proposal.max_vote_bump))
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/5GaPbANXt9dN7Fw6KTvMJxebFbyQorLyT9pijUh9AK9qzNti1qqje2CHXvoFRzMgfW5sSBpWAihjeDK1N51Yq1Tm?cluster=devnet'

In [24]:
accounts = [proposal_pda_meta]

transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([4] + create_num_cnt(Proposal.max_vote_bump))
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/5AsJ2mFYKo9tfWF7pJL4NPSp3HD38CUsDNpRFVKkwwbWHxT82QLYYwpKzjTNhhSHq8Jpw2spzubTxQF5GJi6BeaK?cluster=devnet'